# Download entry IDs

We start with downloading entry IDs for the dataset of AA-M<sup>2+</sup> interaction energies for the further processing:

In [1]:
# imports
import requests
import csv
from tqdm import tqdm

In [2]:
# output file
path_out = 'data/entry_IDs.csv'

# NOMAD Search API URL
url = 'https://nomad-lab.eu/prod/v1/api/v1/entries/query'

# Creating query
query = {
    "datasets.dataset_name:any": [
        "Cation-coordinated conformers of 20 proteinogenic amino acids with different protonation states"
    ]
}

# search parameters
required = {
    "results": {'material': {'chemical_formula_reduced': '*'}},
    "metadata": {'mainfile': '*'},
    'entry_id': '*',
    'upload_id': '*'
}

In [3]:
# Creating .csv file and writing there required data
# The result - list with unique ID, formula and configuration of each structure in the existing database

# start parameters
page_after_value = None
pbar = tqdm()

# writing file
with open(path_out, 'w', newline='') as file:

    # header
    writer = csv.writer(file)
    writer.writerow(['Mainfile', 'Formula', 'Entry_ID', 'Upload_ID'])

    while True:
        # request
        request = {
            'query': query,
            'required': required,
            'pagination': {'page_after_value': page_after_value}
        }
        response = requests.post(url, json = request)

        # check response data
        data = response.json()
        if len(data['data']) == 0:
            break

        # save data
        for entry in data['data']:
            mainfile = entry['mainfile']
            formula = entry['results']['material']['chemical_formula_reduced']
            entry_id = entry['entry_id']
            upload_id = entry['upload_id']
            writer.writerow([mainfile, formula, entry_id, upload_id])

        # update pagination
        page_after_value = data['pagination'].get('next_page_after_value', None)
        if not page_after_value:
            break
        
        # update progress bar
        pbar.update(1)

print('\nDownload is completed.')

2324it [34:56,  1.61it/s]


Download is completed.
